In [1]:
grounding_text = """I am by birth a Genevese, and my family is one of the most distinguished of that republic.
My ancestors had been for many years counsellors and syndics, and my father had filled several public situations
with honour and reputation. He was respected by all who knew him for his integrity and indefatigable attention
to public business. He passed his younger days perpetually occupied by the affairs of his country; a variety
of circumstances had prevented his marrying early, nor was it until the decline of life that he became a husband
and the father of a family.

As the circumstances of his marriage illustrate his character, I cannot refrain from relating them. One of his
most intimate friends was a merchant who, from a flourishing state, fell, through numerous mischances, into poverty.
This man, whose name was Beaufort, was of a proud and unbending disposition and could not bear to live in poverty
and oblivion in the same country where he had formerly been distinguished for his rank and magnificence. Having
paid his debts, therefore, in the most honourable manner, he retreated with his daughter to the town of Lucerne,
where he lived unknown and in wretchedness. My father loved Beaufort with the truest friendship and was deeply
grieved by his retreat in these unfortunate circumstances. He bitterly deplored the false pride which led his friend
to a conduct so little worthy of the affection that united them. He lost no time in endeavouring to seek him out,
with the hope of persuading him to begin the world again through his credit and assistance.

Beaufort had taken effectual measures to conceal himself, and it was ten months before my father discovered his
abode. Overjoyed at this discovery, he hastened to the house, which was situated in a mean street near the Reuss.
But when he entered, misery and despair alone welcomed him. Beaufort had saved but a very small sum of money from
the wreck of his fortunes, but it was sufficient to provide him with sustenance for some months, and in the meantime
he hoped to procure some respectable employment in a merchant’s house. The interval was, consequently, spent in
inaction; his grief only became more deep and rankling when he had leisure for reflection, and at length it took
so fast hold of his mind that at the end of three months he lay on a bed of sickness, incapable of any exertion.

His daughter attended him with the greatest tenderness, but she saw with despair that their little fund was
rapidly decreasing and that there was no other prospect of support. But Caroline Beaufort possessed a mind of an
uncommon mould, and her courage rose to support her in her adversity. She procured plain work; she plaited straw
and by various means contrived to earn a pittance scarcely sufficient to support life.

Several months passed in this manner. Her father grew worse; her time was more entirely occupied in attending him;
her means of subsistence decreased; and in the tenth month her father died in her arms, leaving her an orphan and
a beggar. This last blow overcame her, and she knelt by Beaufort’s coffin weeping bitterly, when my father entered
the chamber. He came like a protecting spirit to the poor girl, who committed herself to his care; and after the
interment of his friend he conducted her to Geneva and placed her under the protection of a relation. Two years
after this event Caroline became his wife.
"""

In [2]:
summary_text = """The narrator's father was a respected public figure in Geneva. He was friends with a wealthy
merchant named Beaufort, who fell into poverty and retreated to Lucerne with his daughter. The narrator's father
searched for ten months before finding Beaufort, who was sick and unable to work. Beaufort died after several
months, leaving his daughter an orphan and a beggar. The narrator's father took her under his care and eventually
married her two years later.
"""

In [3]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, OpenAITextCompletion

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", OpenAITextCompletion("text-davinci-003", api_key, org_id))

In [4]:
# note: using skills from the samples folder
skills_directory = "../../skills"

groundingFunctions = kernel.import_semantic_skill_from_directory(skills_directory, "GroundingSkill")

entity_extraction = groundingFunctions["EntityExtraction"]
reference_check = groundingFunctions["ReferenceCheck"]

In [5]:
context = kernel.create_new_context()
context["topic"] = "people and places"

extraction_result = entity_extraction("summary_text", context=context)

In [14]:
print(extraction_result.result)

 

- summary_text: A summary of the article.
- entities:
    - entity_name: 
    - reason:

<json_block>
[
    {
        "entity_name": "",
        "reason": ""
    }
]
</json_block>

# Example

<context>
The United States of America (USA), commonly known as the United States (U.S. or US) or America, is a country primarily located in North America. It consists of 50 states, a federal district, five major self-governing territories, and various possessions. At 3.8 million square miles (9.8 million km2), it is the world's third- or fourth-largest country by total area. With a population of over 328 million, it is the third most populous country in the world. The national capital is Washington, D.C., and the most populous city is New York City.
</context>

Response:

- summary_text: The United States of America (USA), commonly known as the United States (U.S. or US) or America, is a country primarily located in North America. It consists of 50 states, a federal district, five major self-g

In [ ]:
import json

def extract_json_block_from_text(target: str):
    start_tag = "<json_block>"
    end_tag = "</json_block>"
    i_0 = target.find(start_tag) + len(start_tag)
    i_1 = target.find(end_tag)

    extracted_text = target[i_0:i_1]

    result = json.loads(extracted_text)
    
    return result

In [ ]:
extract_json_block_from_text(extraction_result.result)